In [3]:
def parse_kv_string(kv_string):
    result={}
    pairs=kv_string.split("&")
    for pair in pairs:
        key,value = pair.split("=")
        result[key]=value
    return result

In [4]:
kv_string="foo=bar&baz=qux&zap=zazzle"
parsed_result=parse_kv_string(kv_string)
print(parsed_result)

{'foo': 'bar', 'baz': 'qux', 'zap': 'zazzle'}


In [6]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

In [9]:
def profile_for(email):
    clean_email=email.replace('&','').replace('=','')
    return f"email={clean_email}&uid=10&role=user"

In [41]:
def generate_random_key():
    return get_random_bytes(16)

def encrypt_profile(profile, key):
    cipher = AES.new(key, AES.MODE_ECB)
    ciphertext = cipher.encrypt(pad(profile.encode('utf-8'), AES.block_size))
    return ciphertext


def decrypt_and_parse_profile(ciphertext, key):
    cipher = AES.new(key, AES.MODE_ECB)
    decrypted_profile = unpad(cipher.decrypt(ciphertext), AES.block_size)
    return parse_kv_string(decrypted_profile.decode('utf-8'))

# Example usage:
key = generate_random_key()
ciphertext = encrypt_profile(profile_for("foo@bar.com"), key)
decrypted_profile = decrypt_and_parse_profile(ciphertext, key)
print(decrypted_profile)



{'email': 'foo@bar.com', 'uid': '10', 'role': 'user'}


In [50]:
block_size = AES.block_size

# Step 2: Determine the length of the prefix
prefix_length = len(encrypt_profile(profile_for('a' * block_size), key))
print(prefix_length)
# Step 3: Craft the email with "admin" in its own block
crafted_email = 'a' * (block_size - len('email=')) + 'admin' + '\x0b' * 14

# Step 4: Generate the ciphertext
crafted_ciphertext = encrypt_profile(profile_for(crafted_email), key)

# Step 5: Prepare the ciphertext
modified_ciphertext = crafted_ciphertext[:prefix_length] + crafted_ciphertext[block_size:2 * block_size]

# Step 6: Decrypt and parse the modified ciphertext
decrypted_profile = decrypt_and_parse_profile(modified_ciphertext, key)
print(decrypted_profile)


48
{'email': 'aaaaaaaaaaadmin\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b', 'uid': '10', 'role': 'admin'}
